In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS fleetops_project.silver;

CREATE OR REPLACE TEMP VIEW vw_trucks_latest AS
SELECT * EXCEPT(rn)
FROM (
    SELECT 
        truck_id, plate, model, manufacture_year, capacity_kg, updated_at,
        ROW_NUMBER() OVER (PARTITION BY truck_id ORDER BY _ingested_at DESC) as rn
    FROM fleetops_project.bronze.trucks
) WHERE rn = 1;

CREATE TABLE IF NOT EXISTS fleetops_project.silver.trucks
USING DELTA
AS SELECT * FROM vw_trucks_latest WHERE 1=0;

MERGE WITH SCHEMA EVOLUTION INTO fleetops_project.silver.trucks t
USING vw_trucks_latest s
ON t.truck_id = s.truck_id
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *;